In [1]:
!nvidia-smi

Wed Jan 18 21:37:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:09:00.0 Off |                  N/A |
|  0%   30C    P8    22W / 350W |  17598MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:41:00.0 Off |                  N/A |
| 73%   

In [2]:
#%%capture
#!pip uninstall -y transformers torch
#!conda install -y pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch
#!conda uninstall -y -c huggingface transformers datasets
#!conda uninstall -y -c huggingface transformers datasets

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import transformers
import datasets

#print(f"Running on transformers v{transformers.version} and datasets v{datasets.version}")

## Imports + variable declarations

In [4]:
import torch
from pathlib import Path
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                          TrainingArguments, Trainer)
from train_utils import TrainWrap

import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm

/home/14cfl/anaconda3/envs/openprompt/lib/python3.10/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [5]:
import os
model_ckpt = 'roberta-large'
#train_data = 'generated_test.json'
run_name = 'selftraining'
try:
    os.mkdir(run_name)
except:
    pass
    #max_len = 50
tw = TrainWrap(model_ckpt)

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

## Getting Definitions

## Load data

In [6]:
# train_implicit = pd.read_csv('data/train.tsv', delimiter='\t')
# train_sbic = pd.read_csv("data/SBIC.v2/SBIC.v2.agg.trn.csv")

# val_implicit = pd.read_csv('data/validation.tsv', delimiter='\t')
# val_sbic = pd.read_csv("data/SBIC.v2/SBIC.v2.agg.dev.csv")
#test_text = pd.read_csv('data/test.tsv', delimiter='\t')
# test_implicit = pd.read_csv('data/test.tsv', delimiter='\t')
# test_sbic = pd.read_csv("data/SBIC.v2/SBIC.v2.agg.tst.csv")
train = pd.read_csv('data/reddit_neg.csv')

test = pd.read_csv('data/test_split.csv')

In [7]:
import re

train['text'] = [re.sub('\u2019', "'", x) for x in train['text']]
train['text'] = [re.sub('[“”]', '"', x) for x in train['text']]
train['text'] = [re.sub(r'\\u[a-f0-9]{4}', "'", x) for x in train['text']]
train['text'] = [re.sub(r'\(https://.*\)', "(URL)", x) for x in train['text']]
train['text'] = [re.sub(r'(u/.*? |@.*? )', "user ", x) for x in train['text']]

train['text'] = [re.sub(r'[\s]{2,}', " ", x) for x in train['text']]


In [8]:
train['text'].iloc[69:420].to_list()

['user Harassment? Is that what your new "How to be a Sheep" guide 2.0 is instructing you to say now. Pay up or be quiet, stop asking others to suffer for what YOU deem moral.\nLet me try : Your comment about me being incoherent, a woman and a full central American, is the most sexist and racist, emotionally abusive assault. You should be censored, canceled, jailed, your bank account should be locked.\nFantasies? Ew, creepo.',
 'AITA for telling my MIL she ruined her sons life?\nMy MIL never liked me. She is racist and thinks I am ruining her family legacy by making mixed kids. She got into a heated argument with me after she stole MORE money(she has done this before) and then put my husband as a dependent on her taxes and fucked up our taxes. She went on to say that I am ruining her sons life by "making light skin baby" and my "white ass should stay away from her and her pure black family" I got pissed and said she ruined his life before I was ever in the picture. She was an awful mot

In [9]:
train.columns

Index(['Unnamed: 0.2', 'Unnamed: 0', 'Unnamed: 0.1', 'text', 'text_type',
       'created_utc', 'subreddit', 'reddit_id', 'post', 'predicted_sentiment',
       'confidence'],
      dtype='object')

In [10]:
from sklearn.model_selection import train_test_split

# train_measuring, test = train_test_split(df, train_size=0.8)
# test_measuring, val_measuring = train_test_split(test, train_size=0.75)

# train_measuring.to_csv('data/train_measuring.csv')
# val_measuring.to_csv('data/val_measuring.csv')
# test_measuring.to_csv('data/test_measuring.csv')

train_measuring = pd.read_csv('data/train_measuring.csv')
val_measuring = pd.read_csv('data/val_measuring.csv')
# test_measuring = pd.read_csv('data/test_measuring.csv')

In [11]:
defs = pd.read_csv('data/definitions.csv')
defs

defs = defs.append({'promptName': 'hate speech', 'name': 'hate_speech_score_sigmoid', 
                    'definition': 'bias-motivated, hostile and malicious language targeted at a person/group because of their actual or perceived innate characteristics, especially when the group is unnecessarily labeled'
                   }, ignore_index=True)
defs = defs.append({'promptName': 'Implicit Hatred', 'name': 'other', 
                    'definition': 'the use of coded or indirect language such as sarcasm, metaphor and circumlocution to disparage a protected group or individual, or to convey prejudicial and harmful views about them'
                   }, ignore_index=True)

# defs = defs.append({'promptName': 'Dehumanization', 'name': 'dehumanize', 
#                     'definition': 'the process of depriving a person or group of positive human qualities.'
#                    }, ignore_index=True)

# defs = defs.append({'promptName': 'Violent language', 'name': 'violence', 
#                     'definition': 'calling for the unlawful exercise of physical force or intimidation by the exhibition of such force on a large group of people.'
#                    }, ignore_index=True)

# defs = defs.append({'promptName': 'Genocide', 'name': 'genocide', 
#                     'definition': 'calling for the deliberate killing of a large group of people.'
#                    }, ignore_index=True)

# defs = defs.append({'promptName': 'Respect', 'name': 'respect', 
#                     'definition': 'a feeling of deep admiration for someone or something elicited by their abilities, qualities, or achievements.'
#                    }, ignore_index=True)
# defs = defs.append({'promptName': 'Insult', 'name': 'insult', 
#                     'definition': 'to speak to or treat with disrespect or scornful abuse.'
#                    }, ignore_index=True)

# defs = defs.append({'promptName': 'Humiliation', 'name': 'humiliate', 
#                     'definition': 'making (someone) feel ashamed and foolish by injuring their dignity and self-respect, especially publicly.'
#                    }, ignore_index=True)

# defs = defs.append({'promptName': 'Superiority', 'name': 'status', 
#                     'definition': 'higher in rank, status, or quality.'
#                    }, ignore_index=True)
# defs = defs.append({'promptName': 'Attack', 'name': 'attack_defend', 
#                     'definition': 'an aggressive and violent action against a person or place, and defense is defined as speak or write in favor of (an action or person); attempt to justify.'
#                    }, ignore_index=True)

/tmp/ipykernel_870951/3107126019.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  defs = defs.append({'promptName': 'hate speech', 'name': 'hate_speech_score_sigmoid',
/tmp/ipykernel_870951/3107126019.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  defs = defs.append({'promptName': 'Implicit Hatred', 'name': 'other',


In [12]:

legal_defs = pd.read_csv('legal definitions.csv')

In [13]:

# _, test_sbic_enc = tw.preprocess_prompt(test_sbic, t='sbic',target='hate_speech_score_sigmoid', defs=defs, include_def=False, id_col='Unnamed: 0')
# _, test_implicit_enc = tw.preprocess_prompt(test_implicit, t='implicit', target=implicit_labels, defs=defs, include_def=False, id_col='id')
# _, test_measuring_enc = tw.preprocess_prompt(test_measuring, t='continuous', target='hate_speech_score_sigmoid', defs=defs, include_def=False, text_name='text', id_col='comment_id')

# val, val_sbic_enc = preproc(val_sbic, offensiveness, offensive_labels, tw)
# val, val_implicit_enc = preproc(val_implicit, incitement, incitement_labels, tw)


In [14]:
import random

def interleave_datasets(datasets):
    min_size = min([len(d) for d in datasets])
    
    output = []
    for d in datasets:
        for i in range(min_size):
            output.append(d[i])
            
    return output

In [15]:
import os
import pickle, random

from sklearn.model_selection import train_test_split

dataset_file = 'data/selftraining.pkl'

if not os.path.exists(dataset_file):
    implicit_labels = ['white_grievance','incitement', 'inferiority', 'stereotypical', 'irony', 'threatening', 'other']

    sbic_labels = ['offensiveYN', 'intentYN', 'whoTarget']
    measuring_labels = ['dehumanize', 'genocide', 'violence', 'respect', 'insult', 'humiliate', 'status', 'attack_defend']

    legal_labels = legal_defs['name'].to_list()

#     _, train_sbic_enc = tw.preprocess_prompt(train_sbic, t='continuous', target=sbic_labels, defs=defs, include_def=False, id_col='Unnamed: 0')
#     _, train_implicit_enc = tw.preprocess_prompt(train_implicit, t='implicit', target=implicit_labels, defs=defs, include_def=False, id_col='id')
#     _, train_measuring_enc = tw.preprocess_prompt(train_measuring, t='continuous', target='hate_speech_score_sigmoid', defs=defs, include_def=False, text_name='text', id_col='comment_id')

#     _, val_sbic_enc = tw.preprocess_prompt(val_sbic, t='continuous', target=sbic_labels, defs=defs, include_def=False, id_col='Unnamed: 0')
#     _, val_implicit_enc = tw.preprocess_prompt(val_implicit, t='implicit', target=implicit_labels, defs=defs, include_def=False, id_col='id')
#     _, val_measuring_enc = tw.preprocess_prompt(val_measuring, t='continuous', target='hate_speech_score_sigmoid', defs=defs, include_def=True, text_name='text', id_col='comment_id')

#     train_enc = interleave_datasets([train_sbic_enc, train_implicit_enc, train_measuring_enc
#                                     ])
#     train_enc = train_measuring_enc[:92064]
#     val_enc = interleave_datasets([val_sbic_enc, val_implicit_enc, val_measuring_enc
#                                   ])

#     val_enc = val_measuring_enc[:92064]
    #test_enc = interleave_datasets([test_sbic_enc, test_implicit_enc, test_measuring_enc])

    _, train_enc = tw.preprocess_prompt(train, t='finegrained', target=legal_labels, defs=legal_defs, include_def=True, text_name='text', id_col='reddit_id', use_labels=False)
    
    _, test_enc = tw.preprocess_prompt(test, t='finegrained', target=legal_labels, defs=legal_defs, include_def=True, text_name='post', id_col='id')


    dataset = {}
    dataset['train'] = train_enc[:10000]
    dataset['test'] = test_enc

    with open(dataset_file, 'wb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
        
else:
    with open(dataset_file, 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
        dataset = pickle.load(f)



In [16]:
dataset['train'][0]

{
  "guid": "t1_hxvzclk",
  "label": -1,
  "meta": {
    "definition": "Every person who advocates or promotes genocide is guilty of an indictable offence and liable to imprisonment for a term of not more than five years. In this section, genocide means any of the following acts committed with intent to destroy in whole or in part any identifiable group, namely, killing members of the group; or deliberately inflicting on the group conditions of life calculated to bring about its physical destruction. An intent to directly prompt or provoke another person to commit genocide is enough to establish the mens rea of the offence.",
    "target": "CC_318",
    "target_groups": "colour, race, religion, national or ethnic origin, age, sex, sexual orientation, gender identity or expression, mental or physical disability"
  },
  "text_a": "This guy wouldn't know the high road if it were his neighbor who kicked his ass",
  "text_b": "",
  "tgt_text": null
}

In [17]:

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange
import sys
from MetaST.sampler import UncertaintyDatasetSampler


from MetaST.data_utils import load_and_cache_examples, get_labels
from MetaST.model_utils import multi_source_label_refine, soft_frequency,  opt_grad
#from MetaST.eval import evaluate as meta_eval
from MetaST.modeling_meta_bert import update_parameters, functional_bert

def update_sampling_weight(
                           model,
                           eval_dataset,
                           meta_sampler, 
                           local_rank=-1, train_batch_size=8, device='cuda'):

    eval_sampler = RandomSampler(eval_dataset) if local_rank == -1 else DistributedSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=train_batch_size)
    model.eval()
    for step, eval_batch in enumerate(eval_dataloader):
        eval_batch = tuple(t.to(args.device) for t in eval_batch)
        eval_inputs = {"input_ids": eval_batch[0], "attention_mask": eval_batch[1],
                       "labels": eval_batch[3]}

        outputs = model(**eval_inputs)
        loss, logits, final_embeds = outputs[0], outputs[1], outputs[2]
        meta_sampler.async_update_matrix(eval_batch[-1], eval_batch[3], logits)
    model.train()
    
def meta_selection(args,
                   config,
                   model,
                   optimizer,
                   input_list,
                   meta_dataloader,
                   sampling_step=1):

    if sampling_step < 0:
        sampling_step = len(meta_dataloader)
    la_inputs = input_list[0]
    un_inputs = input_list[1]

    for i in range(sampling_step):

        eval_batch_nocuda = next(meta_dataloader.__iter__())
        eval_batch = tuple(t.to(args.device) for t in eval_batch_nocuda)
        eval_inputs = {"input_ids": eval_batch[0], "attention_mask": eval_batch[1],
                       "labels": eval_batch[3]}
        self_training_fast_model = model
        self_training_fast_model.eval()

        outputs = self_training_fast_model(**un_inputs, reduce=False)
        loss, logits, final_embeds = outputs[0], outputs[1], outputs[2]
        active_loss = outputs[-1]

        if args.use_labeled_data == 1:
            la_outputs = self_training_fast_model(**la_inputs)
            la_loss = la_outputs[0]
            if args.n_gpu > 1:
                la_loss = la_loss.mean()

        elif args.use_labeled_data == 2:
            la_outputs = self_training_fast_model(**la_inputs, reduce=False)
            la_loss = la_outputs[0]

        else:
            la_loss = 0

        if args.self_training_label_mode == "hard":

            if args.use_token_weight == 1:
                if args.use_labeled_data == 2:
                    combined_loss = torch.cat([loss, la_loss], dim=0)
                    weight = torch.zeros(combined_loss.size(), requires_grad=True).to(args.device)
                    new_loss = (combined_loss * weight).sum()
                else:
                    weight = torch.zeros(loss.size(), requires_grad=True).to(args.device)
                    new_loss = (loss * weight).sum() + args.labeled_beta * la_loss
            else:
                weight = torch.zeros(logits.size(0), requires_grad=True).to(args.device)
                sen_weight = weight.unsqueeze(-1).repeat(1, logits.size(1))
                sen_weight = sen_weight.view(-1)[active_loss]
                new_loss = (loss * sen_weight).sum() + args.labeled_beta * la_loss

        else:
            weight = torch.zeros(loss.sum(-1).size(), requires_grad=True).to(args.device)
            new_loss = (loss.sum(-1) * weight).sum() + args.labeled_beta * la_loss

        try:
            fast_weights = update_parameters(self_training_fast_model, new_loss, step_size=args.controler_step_size)
        except:
            pdb.set_trace()
        outputs = functional_bert(fast_weights, config, **eval_inputs, args=args)
        loss, logits, final_embeds = outputs[0], outputs[1], outputs[2]


        if args.n_gpu > 1:
            loss = loss.mean()

        grad_eps = torch.autograd.grad(loss, weight, only_inputs=True)[0]

        # Line 11 computing and normalizing the weights
        if i == 0:
            loss_weight = -grad_eps
        else:
            loss_weight -= grad_eps
    self_training_fast_model.zero_grad()

    loss_weight = torch.clamp(loss_weight, min=0)

    if args.use_token_weight != 1:
        loss_weight = loss_weight.unsqueeze(-1).repeat(1, logits.size(1))
        loss_weight = loss_weight.view(-1)[active_loss]

    norm_c = torch.sum(loss_weight) + 0.0000001

    if norm_c != 0:
        loss_weight = loss_weight / norm_c
    else:
        loss_weight = loss_weight

    model.train()

    return loss.data, loss_weight.detach()

## Load model

In [18]:
# %% [markdown]
# ## Text Classification with LM-BFF.
# In this tutorial, we do sentiment analysis with automatic template and verbalizer generation. We use SST-2 as an example.

# %% [markdown]
# ### 1. load dataset

# %%
# import argparse
# parser = argparse.ArgumentParser("")
# parser.add_argument("--lr", type=float, default=5e-5)
# args = parser.parse_args()

# %% [markdown]
# ### 2. build initial verbalizer and template
# - note that if you wish to do automatic label word generation, the verbalizer is not the final verbalizer, and is only used for template generation.
# - note that if you wish to do automatic template generation, the template text may desirably include `{"meta":"labelword"}` so that label word can be used and remember to use `LMBFFTemplateGenerationTemplate` class so that "labelword" can be handled properly. Else you can just use `ManualTemplate`
# - below is a template that expects plain text generation at each "mask" token position

# %%




In [19]:
# %%
# train util function
# code taken from openprompt with the following modifications:
# 1. evaluates the model every 500 steps, saves the best model per epoch
# 2. saves the model every epoch to easily resume from checkpoint
# 3. logs f1, precision, recall, and loss
# 4. fit returns the best overall model from training


from openprompt.plms import load_plm
from openprompt.prompts.prompt_generator import T5TemplateGenerator
from openprompt.pipeline_base import PromptDataLoader, PromptForClassification
from openprompt.prompts import ManualTemplate
from openprompt.trainer import ClassificationRunner
import copy
import torch
from transformers import  AdamW, get_linear_schedule_with_warmup
from collections import Counter


from sklearn.metrics import f1_score, recall_score, precision_score, hamming_loss


def fit(model, train_dataloader, val_dataloader, loss_func, optimizer, scheduler, resume=False, epochs=10):
    best_score = 0.0
    best_model = model
    last_epoch = 0
    if resume:
        checkpoints = os.listdir(f'{run_name}')
        last_epoch = max([int(re.sub(r'[^0-9]', '', x)) for x in checkpoints if 'epoch' in x])
        
        model.load_state_dict(torch.load(f'{model_path}/epoch-{last_epoch}'))
    
    
    for epoch in tqdm(range(last_epoch, epochs)):
        score, best_model, model = train_epoch(model, train_dataloader, loss_func, optimizer, scheduler, val_dataloader)
        metrics = evaluate(model, val_dataloader)
        
        wandb.log(metrics)
        
        torch.save(model.state_dict(), '{1}/epoch-{0}-{2}'.format(epoch, run_name, score))
        if score > best_score:
            best_score = score
            best_model = model
    return best_score, best_model


def train_epoch(model, train_dataloader, loss_func, optimizer, scheduler, val_dataloader, return_loss=False):
    
    best_score = 0.0
    best_model = model
    model.train()
    batch_loss = []
    prev_loss = 0
    for step, inputs in tqdm(enumerate(train_dataloader)):
        if cuda:
            inputs = inputs.cuda()
        logits = model(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        if return_loss:
            loss_diff = prev_loss - loss.detach()
            batch_loss.append(loss_diff)
        if step % 500 == 0:
            metrics = evaluate(model, val_dataloader, loss_func)
            wandb.log(metrics)
            score = metrics['loss']
            model.train()
            if score > best_score:
                best_score = score
                best_model = model
    if return_loss:
        return best_score, best_model, model, batch_loss
    return best_score, best_model, model

# def test(model, test_dataloader, loss_func=None):
#     results = evaluate(model, test_dataloader, loss_func)
#     labs = [1 if x > 1 else 0 for x in results['labels']]
#     preds = [1 if x > 1 else 0 for x in results['predictions']]

#     print([Counter(labs), Counter(preds)])
#     print([results['micro-f1'], f1_score(labs, preds), recall_score(labs, preds), precision_score(labs, preds)])

def infer(model, val_dataloader, loss_func=None):
    model.eval()
    allpreds = []
    
    batch_loss = []
    all_logits = []
    allloss = 0
    with torch.no_grad():
        for step, inputs in tqdm(enumerate(val_dataloader)):
            if cuda:
                inputs = inputs.cuda()
            logits = model(inputs)
            all_logits.append(logits)
            allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())
        if loss_func is not None:
            loss = loss_func(logits, labels)
            allloss += loss.detach()
            batch_loss.append(loss.detach())
            
    if allloss != 0:
        allloss = allloss/len(val_dataloader)
    
    return {
            'loss': allloss,
            'predictions': allpreds,
            'batch_loss': batch_loss,
            'logits': all_logits
           }
    

def evaluate(model, val_dataloader, loss_func=None):
    model.eval()
    allpreds = []
    alllabels = []
    allloss = 0
    with torch.no_grad():
        for step, inputs in tqdm(enumerate(val_dataloader)):
            if cuda:
                inputs = inputs.cuda()
            logits = model(inputs)
            labels = inputs['label']
            alllabels.extend(labels.cpu().tolist())
            allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())
        if loss_func is not None:
            loss = loss_func(logits, labels)
            #print(logits, labels)
            allloss += loss.detach()
            
    if allloss != 0:
        allloss = allloss/len(val_dataloader)
    
    #print(allpreds, alllabels)
    acc = sum([int(i==j) for i,j in zip(allpreds, alllabels)])/len(allpreds)
    return {'acc': acc, 
            'macro-f1': f1_score(alllabels, allpreds, average='macro'), 
            'micro-f1': f1_score(alllabels, allpreds, average='micro'), 
            'p': precision_score(alllabels, allpreds, average='macro'), 
            'micro-p': precision_score(alllabels, allpreds, average='micro'), 
            'micro-r': recall_score(alllabels, allpreds, average='micro'), 
            'r': recall_score(alllabels, allpreds, average='macro'), 
            'loss': allloss,
            'labels': alllabels,
            'predictions': allpreds,
           }





In [20]:
import wandb
wandb.init(project="lm-bff", name=run_name)

wandb: Currently logged in as: chufeiluo (biases). Use `wandb login --relogin` to force relogin


In [21]:
# %% [markdown]
# ### 4. main training loop

from torch.optim.lr_scheduler import LinearLR
# %%
# main training loop
# train_dataloader = PromptDataLoader(dataset['train'], template, tokenizer=tokenizer, tokenizer_wrapper_class=WrapperClass, batch_size=8)
# valid_dataloader = PromptDataLoader(dataset['val'], template, tokenizer=tokenizer, tokenizer_wrapper_class=WrapperClass, batch_size=8)

mapping = {-1: 1, 0: 0, 1: 2}

test['labels_orig'] = test['labels']

test['labels'] = [mapping[x] for x in test['labels']]


In [22]:
import torch.nn as nn

m = nn.Sigmoid()
model_path = 'fewshot'

def softmax(pred):
    print(pred)
    if len(pred) > 1:
        m = np.max(pred,axis=1,keepdims=True) #returns max of each row and keeps same dims
    else:
        m = np.max(pred)
    
    e_x = np.exp(pred - m) #subtracts each row with its max value
    s = np.sum(e_x,axis=1,keepdims=True) #returns sum of each row and keeps same dims
    f_x = e_x / s 
    
    pred_round = np.round(f_x)
    return pred_round
    #print(pred_round, ep.label_ids)
    
def run_test(model, test_dataloader, test_df):
    num_defs = 11
    results = evaluate(model, test_dataloader)
    
    labs_fine = [1 if x > 1 else 0 for x in results['labels']]
    # preds = [1 if x > 1 else 0 for x in results['predictions']]
    labs_fine = []
    preds = []

    for i in range(num_defs):
        start = i*len(test)
        end = start+len(test)
        labs_fine.append([1 if x > 1 else 0 for x in results['labels'][start:end]])
        preds.append( results['predictions'][start:end])

    labs_coarse = [[x for x in test['labels']]]*11
    for i in range(len(labs_fine)):
        labs_fine[i] = [(mapping[labs_fine[i][j]] if labs_coarse[i][j] != 1 else 1) for j in range(len(labs_coarse[0]))]
        
    res_coarse = []
    res_fine = []


    for i in range(num_defs):
        res_coarse.append((f1_score(labs_coarse[i], preds[i], average='macro'), precision_score(labs_coarse[i], preds[i], average='macro'), recall_score(labs_coarse[i], preds[i], average='macro')))

        res_fine.append((f1_score(labs_fine[i], preds[i], average='macro'), 
                    precision_score(labs_fine[i], preds[i], average='macro'), 
                    recall_score(labs_fine[i], preds[i], average='macro')))
        
    
    print(res_coarse, res_fine)

    return {
        'f1_coarse': np.mean([x[0] for x in res_coarse]),
        'p_coarse': np.mean([x[1] for x in res_coarse]),
        'r_coarse': np.mean([x[2] for x in res_coarse]),
        'f1_fine': np.mean([x[0] for x in res_fine]),
        'p_fine': np.mean([x[1] for x in res_fine]),
        'r_fine': np.mean([x[2] for x in res_fine]),
    }


def self_train(model, dataset, test_dataloader, test_df, num_rounds=5):
    best_score = 0.0
    best_model = model
    last_epoch = 0
    batch_size=8
    
    sorted_conf = None

    # initial labels
    train_dataloader = PromptDataLoader(dataset['train'], template, tokenizer=tokenizer, tokenizer_wrapper_class=WrapperClass, batch_size=batch_size)
    metrics = infer(model, train_dataloader)

    labs = metrics['predictions']
    confidence = [torch.max(m(x), dim=1).values for x in metrics['logits']]
    
    print(Counter(labs))
    
    conf_avg = [x for y in confidence for x in y]

    sorted_test = sorted([(x,y) for x, y in enumerate(conf_avg)], key=lambda x: x[1], reverse=True)

    train_easy = []
    # take 2500 samples teacher model has most confidence in
    for x in sorted_test[:2500]:
        temp = dataset['train'][x[0]]
        temp.label = labs[x[0]]
        train_easy.append(temp)
    
    train_dataloader = PromptDataLoader(train_easy, template, tokenizer=tokenizer, tokenizer_wrapper_class=WrapperClass, batch_size=8)
    loss_func = torch.nn.CrossEntropyLoss()
    no_decay = ['bias', 'LayerNorm.weight']
    # it's always good practice to set no decay to biase and LayerNorm parameters
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=5e-6)
    # scheduler  = LinearLR(optimizer)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=500, num_training_steps=epochs*len(train_dataloader))
#     if cuda:
#         model = model.cuda()

    
    # self training loop
    for r in range(num_rounds):
        for e in range(1):
            score, best_model, model, batch_loss = train_epoch(model=model, train_dataloader=train_dataloader, val_dataloader=test_dataloader, loss_func=loss_func, optimizer=optimizer, scheduler=scheduler, return_loss=True)
            print(score)

            metrics = run_test(model, test_dataloader, test_df)

            wandb.log(metrics)

            torch.save(model.state_dict(), '{1}/run-{0}-{2}'.format(r, run_name, e))
#             if score > best_score:
#                 best_score = score
#                 best_model = copy.deepcopy(model)
        
        
        # re-order batches by easiest to hardest
        if sorted_conf is None:
            sorted_conf = sorted([(x,y) for x, y in enumerate(batch_loss)], key=lambda x: x[1])
        else:
            sorted_conf = sorted([(sorted_conf[i][0], batch_loss[i]) for i in range(len(sorted_conf))], key=lambda x: x[1])
        
        train_reordered = []
        train_dataloader = PromptDataLoader(train_easy, template, tokenizer=tokenizer, tokenizer_wrapper_class=WrapperClass, batch_size=8)
    
        metrics = infer(model, train_dataloader)

        labs = metrics['predictions']
        print(Counter(labs))
        for i in range(len(train_easy)):
            train_easy[i].label = labs[i]
            
        for x in sorted_conf:
            start = x[0]*batch_size
            end = start+batch_size
            train_reordered.extend(train_easy[start:min(end, len(train_easy))])
        #print(train_reordered)
        train_dataloader = PromptDataLoader(train_reordered, template, tokenizer=tokenizer, tokenizer_wrapper_class=WrapperClass, batch_size=8)

    return best_score, best_model

In [23]:
print('load model...')
from openprompt.plms import load_plm
# load mlm model for main tasks
plm, tokenizer, model_config, WrapperClass = load_plm("roberta", "roberta-large")

# load generation model for template generation
# template_generate_model, template_generate_tokenizer, template_generate_model_config, template_tokenizer_wrapper = load_plm('t5', 't5-large')

from openprompt.prompts import ManualVerbalizer, ManualTemplate
verbalizer = ManualVerbalizer(tokenizer=tokenizer, num_classes=3, label_words=[['No'], ['Maybe'], ['Yes']])

from openprompt.prompts.prompt_generator import LMBFFTemplateGenerationTemplate
#template = LMBFFTemplateGenerationTemplate(tokenizer=template_generate_tokenizer, verbalizer=verbalizer, text='{"placeholder":"text_a"}\n{"mask"}.')
template = ManualTemplate(tokenizer=tokenizer, text='{"meta":"target"} is defined as {"meta": "definition"}.\n Post:{"placeholder":"text_a"} Q: Is this post a violation of {"meta":"target"}? Please answer yes or no. A:{"mask"}')

# view wrapped example
wrapped_example = template.wrap_one_example(dataset['train'][0])
print(wrapped_example)

# %%
# parameter setting
cuda = True
auto_t = False # whether to perform automatic template generation
auto_v = False # whether to perform automatic label word generation
epochs=10


load model...
[[{'text': 'CC_318', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': ' is defined as', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': ' Every person who advocates or promotes genocide is guilty of an indictable offence and liable to imprisonment for a term of not more than five years. In this section, genocide means any of the following acts committed with intent to destroy in whole or in part any identifiable group, namely, killing members of the group; or deliberately inflicting on the group conditions of life calculated to bring about its physical destruction. An intent to directly prompt or provoke another person to commit genocide is enough to establish the mens rea of the offence.', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': '.\n Post:', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': "This guy wouldn't know the high road if it were his neighbor who kicked his ass", 'loss_ids': 0, 'shortenable_ids': 1}, {'text': ' Q: Is this post a violation of', 'loss_ids': 0, 's

In [24]:

test_dataloader = PromptDataLoader(dataset['test'], template, tokenizer=tokenizer, tokenizer_wrapper_class=WrapperClass, batch_size=8)


tokenizing: 6281it [00:10, 626.69it/s]


In [25]:

model = PromptForClassification(copy.deepcopy(plm), template, verbalizer)

model.load_state_dict(torch.load(f'{model_path}/best'))

if cuda:
    model.cuda()

In [ ]:
self_train(model, dataset, test_dataloader, test)

tokenizing: 269it [00:00, 677.29it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (695 > 512). Running this sequence through the model will result in indexing errors
tokenizing: 10000it [00:17, 578.61it/s]


0it [00:00, ?it/s]

Counter({0: 9771, 2: 229})


tokenizing: 2500it [00:03, 708.06it/s]
/home/14cfl/anaconda3/envs/openprompt/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0it [00:00, ?it/s]

0it [00:00, ?it/s]

/home/14cfl/anaconda3/envs/openprompt/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
test